# Load Modules

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from plotly.subplots import make_subplots
from multiprocessing import Pool
from tqdm import tqdm
import networkx as nx
import os
import gzip
import glob


pio.templates.default = 'plotly_white'
pd.options.mode.chained_assignment = None

# Load Data

## Define Functions

In [2]:
def seqReader(fn):
    """
    iterate through sequences and yield as generator
    """
    def openSeq(fn):
        if 'gz' in fn:
            return gzip.open(fn, 'rt')
        else:
            return open(fn, 'r')

    def num_iter(fn):
        if 'fastq' in fn or 'fq' in fn:
            return 4
        else:
            return 2

    n = num_iter(fn)

    with openSeq(fn) as f:
        while True:
            try:
                yield [next(f).strip('\n') for _ in range(n)]
            except StopIteration:
                break


## Define Paths

In [3]:
# define directory paths
input_dir = "../results/global_enriched/"
motif_dir = os.path.join(input_dir, "motifs")
clique_dir = os.path.join(input_dir, "cliques")

# input data paths
conv_fn = os.path.join(input_dir, "global_enriched.conv.txt")
motif_seq_fn = os.path.join(motif_dir, "merged_motifs.fa")
motif_blast_fn = os.path.join(motif_dir, "motifs.blast.tab")
merged_cliques = os.path.join(clique_dir, "merged_cliques.tab")

## Process Motifs

In [4]:
# load fasta into dataframe
motif_frame = pd.DataFrame([
    [h.strip('>'), s] for h, s in seqReader(motif_seq_fn)
    ],
    columns = ["qseqid", "sequence"]
)

# load blast results into frame
motif_blast = pd.read_csv(motif_blast_fn, sep="\t")
minimum_evalue = motif_blast.\
    groupby('qseqid').\
    apply(
        lambda x : pd.Series({
            'sseqid' : x.sort_values('evalue').iloc[0].sseqid,
            'evalue' : x.sort_values('evalue').iloc[0].evalue
        })
    ).reset_index()


# create clique idx variable
minimum_evalue['clique_idx'] = minimum_evalue.qseqid.\
    apply(lambda x : x.split("_")[1]).\
    astype(int)

motif_frame = motif_frame.merge(minimum_evalue)
motif_frame

,qseqid,sequence,sseqid,evalue,clique_idx
0,clique_0_MEME-1,VIPEELVEEVIP,eid_11776::11-1_polypeptide,0.380000,0
1,clique_0_MEME-2,VVEEVVPEELVE,eid_914212::glutamate-rich_protein_[Plasmodium...,8.800000,0
2,clique_0_MEME-3,EVVEEVVPE,eid_11776::11-1_polypeptide,4.700000,0
3,clique_1_MEME-1,GIDJYBEVLKRK,eid_73540::PfEMP1_variant_1_of_strain_MC,0.470000,1
4,clique_1_MEME-10,ENELFGT,eid_913468::heat_shock_protein_90,2.100000,1
...,...,...,...,...,...
170,clique_92_MEME-1,PCZLEYQWHTNV,eid_236266::erythrocyte_membrane_protein_1,0.000076,92
171,clique_94_MEME-1,NYHYEEPFILTP,eid_1068229::high_molecular_weight_rhoptry_pro...,2.700000,94
172,clique_94_MEME-2,EAYEKAGEVIVE,eid_23729::merozoite_surface_protein_3,3.800000,94
173,clique_94_MEME-3,REVEKRSEKLID,eid_15297::sporozoite_surface_protein_2,1.500000,94


## Create Conversion Table

In [5]:
f = open(conv_fn, "r+")
conv_dict = {
    line.strip().split(' ')[0] : line.strip().split(' ')[1] for line in f
}

## Load Graph from Edgelist

In [6]:
G = nx.read_edgelist(merged_cliques)
G = nx.relabel_nodes(G, conv_dict)

## Create Clique Lookup Table

In [7]:
clique_table = []
for fn in glob.glob("{}/*fa".format(clique_dir)):
    clique_idx = int(fn.split("/")[-1].split(".")[0].split("_")[1])
    
    
    for header, seq in seqReader(fn):
        clique_table.append({
            'clique_idx' : clique_idx,
            "header" : header.strip(">"),
            "seq" : seq
        })
        

clique_table = pd.DataFrame(clique_table)
clique_table

,clique_idx,header,seq
0,233,t214801,HLRDAGGNKIGPPASHAIPQMINNLVGEATQGAAEVAKKASESATA...
1,233,t215202,ENVNTTITGNDFSGGEFLWPGYTEELKAKKASEDAEKAANDAENAS...
2,289,t118859,TIEENQNNELEGTFKKLIVVVKELSDKNKELDEKEKKIKTYNGDIQ...
3,289,t149561,KFFYKKVENTKNKYMNKKKFSTKSEDIINKNNNTTKGSTLGGENDL...
4,214,t113780,EPNENSVVDRATDSMNLDPEKVHNENMSDPNTNTEPDASLKDDKKE...
...,...,...,...
2591,78,t130133,HRNDHRNDHRNDHRNDHRNDHRTDHRNDHRNDHRNDHRNDQRNDHR...
2592,78,t128339,NDQRNDHRNDHRNDHRNDHRNDHRNDHRNDHRNDHRNDHRNDQRND...
2593,78,t130135,NDHRNSMRNDHRNSMRNDQRNSMMNDQRNSMMNDQRNSMMNDQRNV...
2594,234,t209489,SGPGDVSFSSSEKPTLYLDELARPVSKPRPAKQTKSQPVKDLAGQK...


## Aggregate Clique Layouts

In [8]:
layout = nx.spring_layout(G, seed=42)

In [9]:
frame = []
for idx, c in clique_table.groupby('clique_idx'):
    
    header_set = set(c.header.values)
    
    agg_x, agg_y = np.array([
        layout[h] for h in header_set
        ]).\
        mean(axis = 0)
    
    frame.append({
        'clique_idx' : idx,
        'membership' : len(header_set),
        'agg_x' : agg_x,
        'agg_y' : agg_y
    })
    

frame = pd.DataFrame(frame)

In [10]:
group_cols = ['clique_idx', 'membership', 'agg_x', 'agg_y']
clique_motif_frame = []
nan_remover = lambda x : np.array([i for i in x.astype(str) if i != 'nan'])

for idx, subframe in frame.merge(motif_frame, how = 'left').groupby(group_cols):
    
    data = {
        'clique_idx' : idx[0],
        'membership' : idx[1],
        'agg_x' : idx[2],
        'agg_y' : idx[3],
        'qseqid' : nan_remover(subframe.qseqid.unique()),
        'sequences' : nan_remover(subframe.sequence.unique()),
        'sseqid' : nan_remover(subframe.sseqid.unique()),
        'num_known' : nan_remover(subframe.sseqid.unique()).size
    }
    
    clique_motif_frame.append(data)

clique_motif_frame = pd.DataFrame(clique_motif_frame)
clique_motif_frame.sort_values('num_known', inplace=True)
clique_motif_frame

,clique_idx,membership,agg_x,agg_y,qseqid,sequences,sseqid,num_known
292,292,2,0.647536,-0.538006,[],[],[],0
120,120,4,0.251843,-0.825419,[],[],[],0
220,220,2,-0.424170,0.857366,[],[],[],0
219,219,2,-0.817506,-0.318478,[],[],[],0
123,123,4,-0.876185,0.305509,[],[],[],0
...,...,...,...,...,...,...,...,...
16,16,25,-0.363866,-0.045024,"[clique_16_MEME-1, clique_16_MEME-2, clique_16...","[TDNEWNTLKDEF, DIQNDGIPSSKI, ISQYLQSEQPKD, VPN...","[eid_43848::hypothetical_protein_PFL1930w, eid...",3
7,7,38,-0.298488,0.120337,"[clique_7_MEME-1, clique_7_MEME-2, clique_7_ME...","[QQSDLEQERLAK, KLQEQQS, LAKEKLQ]",[eid_913175::liver_stage_antigen_1_[Plasmodium...,3
21,21,21,0.339539,0.108099,"[clique_21_MEME-1, clique_21_MEME-2, clique_21...","[IYYDVNDDD, KEKYPIADVWDI, DKPFNEP]",[eid_1067887::acidic_basic_repeat_antigen_[Pla...,3
8,8,31,-0.119946,-0.324541,"[clique_8_MEME-1, clique_8_MEME-2, clique_8_ME...","[AITKYQNPH, YHNDPELKEIID, NTQRTTIKSRLL, DPYKQL...",[eid_17796::Cytoadherence_linked_asexual_prote...,4


# Plot Cliques and Motifs

## Aggregated Clique Layout + Number of found motifs

In [12]:
clique_motif_frame['joined_sequences'] = [
    ','.join(x) for x in clique_motif_frame.sequences
]


clique_motif_frame['joined_epitopes'] = [
    ','.join([i.split("::")[-1].replace("[Plasmodium_falciparum_3D7]", "") for i in x]) for x in clique_motif_frame.sseqid
]

fig = go.Figure()

tr = go.Scatter(
    x = clique_motif_frame.agg_x,
    y = clique_motif_frame.agg_y,
    mode = 'markers',
    marker = dict(
        size = clique_motif_frame.membership,
        color = clique_motif_frame.num_known,
        sizemode = 'area',
        sizeref=100*max(clique_motif_frame.num_known)/(40.**2),
        sizemin = 3,
        symbol = [0 if x > 0 else 100 for x in clique_motif_frame.num_known],
        colorbar = dict(title = 'known motifs'),
        colorscale = ["#f5d60f", "#C70039"]    
    ),
    customdata = clique_motif_frame[['clique_idx', 'joined_sequences', 'joined_epitopes']].values,
    hovertemplate = 
        '<b>Clique Number</b> : %{customdata[0]}<br>' +
        "<b>Membership</b> : %{marker.size}<br>" +
        '<b>Motifs</b> : %{customdata[1]}</b><br>' +
        "<extra><b>Known Epitopes</b> : %{customdata[2]}<br></extra>",
)

fig.add_trace(tr)


fig.update_layout(
    height = 1000, width = 1000,
    hoverlabel = dict(font=dict(color='white'))
)

fig.write_html("../figures/enriched_set_clique_motifs/clique_scatter.html")
fig

## Distribution of Number of Found Motifs

In [13]:
known_frame = clique_motif_frame[['clique_idx', 'membership', 'num_known']].drop_duplicates()

known_hist = known_frame.\
    groupby('num_known').apply(lambda x : pd.Series({'counts':x.shape[0]})).reset_index()

px.bar(
    known_hist, x = 'num_known', y = 'counts'
)

## Membership size of nodes with number of motifs found

In [14]:
fig = px.box(
    known_frame, x = 'num_known', y = 'membership',
    log_y=True, hover_name='clique_idx'
)

fig.update_layout(height = 1000)

In [15]:
num_unknown_motifs = (known_frame.num_known.astype(int) == 0).sum()
total_cliques = known_frame.clique_idx.unique().size

print(
    "Total Unknown Clique Motifs : {} / {} ( {:.3f} )".\
    format(num_unknown_motifs, total_cliques, num_unknown_motifs/total_cliques)
)

# known_frame.clique_idx.unique().size

Total Unknown Clique Motifs : 168 / 293 ( 0.573 )


In [16]:
clique_motif_frame[
    (clique_motif_frame.num_known == 0) & 
    (clique_motif_frame.membership > 10)
].sort_values("membership")

,clique_idx,membership,agg_x,agg_y,qseqid,sequences,sseqid,num_known,joined_sequences,joined_epitopes
42,42,11,0.622609,-0.371800,[],[],[],0,,
41,41,12,-0.620587,-0.237495,[],[],[],0,,
35,35,13,0.211972,-0.663392,[],[],[],0,,
37,37,13,0.422244,0.224141,[],[],[],0,,
38,38,13,0.535226,0.248323,[],[],[],0,,
39,39,13,-0.520317,-0.477629,[],[],[],0,,
29,29,15,-0.637808,0.192836,[],[],[],0,,
27,27,15,0.105428,-0.585399,[],[],[],0,,
26,26,16,-0.138100,0.359486,[],[],[],0,,
24,24,19,0.055179,0.337752,[],[],[],0,,


# Motif Distribution

In [92]:
import scipy

motif_seq_frame = []

for h, s in seqReader(motif_seq_fn):
    _, clique_idx, motif_id, evalue = h.split("_")
    motif_seq_frame.append({
        "clique_idx" : int(clique_idx),
        "motif_id" : motif_id,
        "evalue" : float(evalue),
        "seq" : s
    })
    
motif_seq_frame = pd.DataFrame(motif_seq_frame).merge(clique_motif_frame[["clique_idx", "membership"]])
motif_seq_frame

,clique_idx,motif_id,evalue,seq,membership
0,0,MEME-0,0.000000e+00,VIPEELVEEVIP,464
1,0,MEME-1,0.000000e+00,VVEEVVPEELVE,464
2,0,MEME-2,0.000000e+00,EVVEEVVPE,464
3,0,MEME-3,1.000000e-67,PEELVEE,464
4,0,MEME-4,5.500000e-53,EELVEEV,464
...,...,...,...,...,...
372,95,MEME-0,3.900000e-27,TINNLYVGDFY,6
373,95,MEME-1,2.400000e-07,QKKEYEK,6
374,97,MEME-0,1.400000e-107,NDKDIYNNM,5
375,98,MEME-0,1.600000e-60,DVDGENKKNGEH,5


## Plot Correlation of membership and discovered motifs

In [93]:
n_motifs_frame = motif_seq_frame.\
    groupby(["clique_idx", "membership"]).\
    apply(lambda x : pd.Series({"n_motifs" : x.shape[0]})).\
    reset_index()

px.scatter(
    n_motifs_frame, x = 'membership', y = "n_motifs",
    log_x = True
)

## Distribution of E-Value with Motif Rank

In [94]:
px.box(
    motif_seq_frame, x = 'motif_id', y = 'evalue',
    points = 'all', log_y = True
)

In [84]:
motif_seq_frame[motif_seq_frame.evalue == 0]

,clique_idx,motif_id,evalue,seq,membership
0,0,MEME-0,0.0,VIPEELVEEVIP,464
1,0,MEME-1,0.0,VVEEVVPEELVE,464
2,0,MEME-2,0.0,EVVEEVVPE,464
5,1,MEME-0,0.0,GIDJYBEVLKRK,169
6,1,MEME-1,0.0,NELFGTNHVKQT,169
7,1,MEME-2,0.0,DIPKYVSNNVYS,169
8,1,MEME-3,0.0,SSYNVAKSTNSD,169
17,10,MEME-0,0.0,RPQFLRWFTEWG,29
35,11,MEME-0,0.0,VEENVEE,28
49,12,MEME-0,0.0,RNQNVNDRRNFD,27


# Response Rate

In [72]:
patient_frame = []
for fn in glob.glob("../data/enriched_fastas/Plate*.txt"):
    cid = fn.split("/")[-1].split("_")[1]
    for header, seq in seqReader(fn):
        patient_frame.append({
            'cid' : cid,
            "header" : header.strip(">")
        })

patient_frame = pd.DataFrame(patient_frame)
patient_frame = patient_frame.merge(clique_table[['clique_idx', 'header']])

In [73]:
number_responses = patient_frame.\
    groupby('clique_idx').\
    apply(
        lambda x : pd.Series({
            'num_patients' : x.cid.unique().size
        })
    ).reset_index()

## Clique Motifs with Patient Responses

In [74]:
clique_response_frame = clique_motif_frame.merge(number_responses)

fig = px.scatter(
    clique_response_frame, x = 'agg_x', y = 'agg_y', size = 'membership',
    size_max = 50, hover_name='clique_idx', hover_data=['qseqid', 'sequences', 'sseqid'],
    color = 'num_patients', color_continuous_scale="OrRd",
    facet_col="num_known",facet_col_wrap=2
)

fig.update_layout(
    height = 1000, width = 1000
)

fig.write_html("../figures/enriched_set_clique_motifs/patient_responses.html")

fig

## Correlation of number of known motifs against patient recognition

In [20]:
clique_responses = clique_response_frame[
    ['clique_idx', 'membership', 'num_known', 'num_patients']
].drop_duplicates()

px.scatter(
    clique_responses, x = 'num_known', y = 'num_patients',
    size = 'membership', color = 'num_known',
    hover_name="clique_idx", size_max=50
)

In [21]:
patient_frame['location'] = patient_frame.cid.apply(lambda x : 'Tororo' if 'CT' in x else 'Kanungu')
patient_response_by_region = patient_frame.\
    groupby(['location', 'clique_idx']).\
    apply(
        lambda x : pd.Series({'num_patients' : x.cid.unique().size})
).reset_index()

patient_response_by_region = pd.pivot_table(
    patient_response_by_region, index = 'clique_idx', 
    columns = 'location', values = 'num_patients'
).reset_index()

patient_response_by_region['leaning'] = patient_response_by_region.apply(
    lambda x : 'K' if x.Kanungu > x.Tororo else 'T',
    axis = 1
)

patient_response_by_region

location,clique_idx,Kanungu,Tororo,leaning
0,0,47,73,T
1,1,17,42,T
2,2,32,64,T
3,3,72,70,K
4,4,54,23,K
...,...,...,...,...
288,288,10,8,K
289,289,12,10,K
290,290,9,13,T
291,291,5,14,T


## Regional Observations of Cliques

In [22]:
fig = px.scatter(
    clique_response_frame.merge(patient_response_by_region),
    x = 'agg_x', y = 'agg_y', size = 'membership',
    size_max = 50, hover_name='clique_idx', hover_data=['qseqid', 'sequences', 'sseqid'],
    facet_row = 'leaning',
    color = 'num_known',
    color_discrete_sequence=['#06021c', '#dcb825', '#dc5c25', '#dc2549']
)

fig.update_layout(
    height = 1000, width = 1000,
    coloraxis_colorbar=dict(
        tickvals = [0,1,2,3]
    )
)

fig.update_layout(height = 1000, width = 1000)